In [6]:
!pip install kaggle opencv-python torch torchvision torchmetrics albumentations matplotlib nibabel tqdm scikit-learn scikit-image

In [12]:
!mkdir -p ~/.kaggle

In [16]:
import os

os.environ['KAGGLE_USERNAME'] = "vanhoai"
os.environ["KAGGLE_KEY"] = "KGAT_f693f7d3a42a54c1aedabf202849dfca"

In [17]:
!kaggle datasets download awsaf49/brats20-dataset-training-validation

Dataset URL: https://www.kaggle.com/datasets/awsaf49/brats20-dataset-training-validation
License(s): CC0-1.0
100% 4.14G/4.16G [00:47<00:00, 35.7MB/s]
100% 4.16G/4.16G [00:47<00:00, 93.9MB/s]


In [18]:
!dir

brats20-dataset-training-validation.zip  sample_data


In [ ]:
!unzip brats20-dataset-training-validation.zip "BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_0[0-5][0-9]/*"

In [19]:
!unzip brats20-dataset-training-validation.zip

Archive:  brats20-dataset-training-validation.zip
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_flair.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1ce.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t2.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002/BraTS20_Training_002_flair.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002/BraTS20_Training_002_seg.nii  
  inflating: BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_002/BraTS20_Training_002_t1.nii  
  in

In [20]:
!dir

BraTS2020_TrainingData	  brats20-dataset-training-validation.zip
BraTS2020_ValidationData  sample_data


In [38]:
ROOT_DIR = "./BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"

In [22]:
# Find min and max values in the array
data_folders = list(os.listdir(ROOT_DIR))
modalities = ["flair", "t1", "t1ce", "t2"]

min_overall = float("inf")
max_overall = float("-inf")

for sampling in modalities:
    min_value = float("inf")
    max_value = float("-inf")
    count = 0

    for folder in data_folders:
        file_path = os.path.join(ROOT_DIR, folder, f"{folder}_{sampling}.nii")
        if not os.path.exists(file_path):
            continue

        image = nib.load(file_path).get_fdata()  # type: ignore
        current_min = image.min()
        current_max = image.max()

        min_value = min(current_min, min_value)
        max_value = max(current_max, max_value)

        count = count + 1

    min_overall = min(min_value, min_overall)
    max_overall = max(max_value, max_overall)

    print("The number of folders processed for ", sampling, ": ", count)
    print("Overall Min value in FLAIR images: ", min_value)
    print("Overall Max value in FLAIR images: ", max_value)

print("Final Overall Min value: ", min_overall)  # -7.943429470062256
print("Final Overall Max value: ", max_overall)  # 32767.0

The number of folders processed for  flair :  369
Overall Min value in FLAIR images:  -7.943429470062256
Overall Max value in FLAIR images:  32767.0
The number of folders processed for  t1 :  369
Overall Min value in FLAIR images:  0.0
Overall Max value in FLAIR images:  32767.0
The number of folders processed for  t1ce :  369
Overall Min value in FLAIR images:  0.0
Overall Max value in FLAIR images:  32767.0
The number of folders processed for  t2 :  369
Overall Min value in FLAIR images:  -2.0040841102600098
Overall Max value in FLAIR images:  32767.0
Final Overall Min value:  -7.943429470062256
Final Overall Max value:  32767.0


In [ ]:
!dir /Brats2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_005/

In [40]:
dir_355 = ROOT_DIR + "/BraTS20_Training_355/"
dir_355

'./BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/'

In [46]:
!dir ./BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/

BraTS20_Training_355_flair.nii	BraTS20_Training_355_t1.nii
BraTS20_Training_355_seg.nii	BraTS20_Training_355_t2.nii
BraTS20_Training_355_t1ce.nii


In [45]:
SAMPLE = "355"
SAMPLE_PATH = ROOT_DIR + f'/BraTS20_Training_{SAMPLE}/'

# os.rename(SAMPLE_PATH + 'W39_1998.09.19_Segm.nii', SAMPLE_PATH + 'BraTS20_Training_355_seg.nii')
os.rename(SAMPLE_PATH + 'BraTS20_Training_355_seg.nii.gz', SAMPLE_PATH + 'BraTS20_Training_355_seg.nii')

In [47]:
from sklearn.preprocessing import MinMaxScaler
import nibabel as nib

scaler = MinMaxScaler()

modalities = ["flair", "t1", "t1ce", "t2", "seg"]
modalities_image = {}

for modality in modalities:
    # Load image with nibabel
    image = nib.load(SAMPLE_PATH + f'BraTS20_Training_{SAMPLE}_{modality}.nii').get_fdata()

    if modality != "seg":
        # Rescale image
        image = scaler.fit_transform(image.reshape(-1, image.shape[-1])).reshape(image.shape)
        modalities_image[modality] = image
    else:
        # Do not rescale segmentation mask
        modalities_image[modality] = image

modalities_image["seg"]

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [27]:
import os
import torch
import numpy as np
import nibabel as nib
from typing import List, Tuple
from torch.utils.data import Dataset
import cv2

SEGMENT_CLASSES = {
    0: "NOT tumor",
    1: "NECROTIC/CORE",  # or NON-ENHANCING tumor CORE
    2: "EDEMA",
    3: "ENHANCING",  # original 4 -> converted into 3
}

# Select Slices and Image Size
VOLUME_SLICES = 100
VOLUME_START_AT = 22  # first slice of volume that we will include
IMG_SIZE = 128


class BrainTumorDataset(Dataset):
    def __init__(
        self,
        ids: List[str],
        data_path: str,
        img_size: int = IMG_SIZE,
        n_channels: int = 2,
        volume_slices: int = VOLUME_SLICES,
        volume_start_at: int = VOLUME_START_AT,
        transform=None,
    ) -> None:
        self.ids = ids
        self.data_path = data_path
        self.img_size = img_size
        self.n_channels = n_channels
        self.volume_slices = volume_slices
        self.volume_start_at = volume_start_at
        self.transform = transform

        # Calculate total number of slices
        self.total_slices = len(self.ids) * self.volume_slices

    def __len__(self) -> int:
        return self.total_slices

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Get one slice of data
        Returns:
            X: Input tensor of shape (n_channels, img_size, img_size)
            Y: Target mask tensor of shape (4, img_size, img_size) - one-hot encoded

        Example:
            idx=0
            patient_idx = 0 // 100 = 0
            slice_idx = 0 % 100 = 0
            -> First slice of first patient
        """

        # Calculate which patient and which slice
        patient_idx = idx // self.volume_slices
        slice_idx = idx % self.volume_slices

        patient_id = self.ids[patient_idx]
        case_path = case_path = os.path.join(self.data_path, patient_id)

        # Load FLAIR
        flair_path = os.path.join(case_path, f"{patient_id}_flair.nii")
        flair = nib.load(flair_path).get_fdata()  # type: ignore

        # Load T1CE
        t1ce_path = os.path.join(case_path, f"{patient_id}_t1ce.nii")
        t1ce = nib.load(t1ce_path).get_fdata()  # type: ignore

        # Load Segmentation mask
        seg_path = os.path.join(case_path, f"{patient_id}_seg.nii")
        seg = nib.load(seg_path).get_fdata()  # type: ignore

        # Extract and resize the specific slice
        actual_slice = slice_idx + self.volume_start_at

        flair_slice = cv2.resize(
            flair[:, :, actual_slice],
            (self.img_size, self.img_size),
        )

        t1ce_slice = cv2.resize(
            t1ce[:, :, actual_slice],
            (self.img_size, self.img_size),
        )

        seg_slice = cv2.resize(
            seg[:, :, actual_slice],
            (self.img_size, self.img_size),
            interpolation=cv2.INTER_NEAREST,  # Use nearest neighbor for labels
        )

        # Stack channels: (H, W, C) -> (C, H, W)
        X = np.stack([flair_slice, t1ce_slice], axis=0).astype(np.float32)

        # Normalize input
        X = X / (np.max(X) + 1e-8)  # Add small epsilon to avoid division by zero

        # Process segmentation mask
        # Convert label 4 to 3
        seg_slice[seg_slice == 4] = 3

        # One-hot encode: (H, W) -> (4, H, W)
        Y = np.zeros((4, self.img_size, self.img_size), dtype=np.float32)
        for c in range(4):
            Y[c] = (seg_slice == c).astype(np.float32)

        # Convert to tensors
        X = torch.from_numpy(X)
        Y = torch.from_numpy(Y)

        # Apply transforms if any
        if self.transform:
            X, Y = self.transform(X, Y)

        return X, Y

    def get_patient_volume(self, patient_idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Get all slices for a specific patient

        Returns:
            X: Input tensor of shape (volume_slices, n_channels, img_size, img_size)
            Y: Target mask tensor of shape (volume_slices, 4, img_size, img_size)
        """
        start_idx = patient_idx * self.volume_slices
        end_idx = start_idx + self.volume_slices

        X_list = []
        Y_list = []

        for idx in range(start_idx, end_idx):
            X, Y = self[idx]
            X_list.append(X)
            Y_list.append(Y)

        X_volume = torch.stack(X_list, dim=0)
        Y_volume = torch.stack(Y_list, dim=0)

        return X_volume, Y_volume

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DiceLoss(nn.Module):
    """
    Dice Loss cho Segmentation
    Dice = 2 * |A ∩ B| / (|A| + |B|)
    """

    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth

    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Args:
            pred: (batch, num_classes, H, W) - logits
            target: (batch, num_classes, H, W) - one-hot encoded
        """
        pred = torch.softmax(pred, dim=1)  # Convert logits to probabilities

        # Flatten
        pred = pred.view(pred.size(0), pred.size(1), -1)  # (batch, classes, H*W)
        target = target.view(target.size(0), target.size(1), -1)

        # Dice score
        intersection = (pred * target).sum(dim=2)
        union = pred.sum(dim=2) + target.sum(dim=2)  # (batch, classes)

        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        dice_loss = 1 - dice.mean()  # Mean across all classes and batch

        return dice_loss

In [29]:
import torch
import torch.nn as nn

class CombinedLoss(nn.Module):
    def __init__(self, ce_weight=0.5, dice_weight=0.5):
        super().__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.ce_loss = nn.CrossEntropyLoss()
        self.dice_loss = DiceLoss()

    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Args:
            pred: (batch, num_classes, H, W) - logits
            target: (batch, num_classes, H, W) - one-hot encoded
        """
        # CrossEntropy Loss expects class indices, not one-hot encoded targets
        target_indices = torch.argmax(target, dim=1)  # (batch, H, W)

        ce = self.ce_loss(pred, target_indices)
        dice = self.dice_loss(pred, target)

        return self.ce_weight * ce + self.dice_weight * dice

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DualConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        mid_channels: int | None = None,
    ):
        super(DualConv, self).__init__()
        if not mid_channels:
            mid_channels = out_channels

        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.sequential(x)


class Down(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(Down, self).__init__()
        self.sequential = nn.Sequential(
            nn.MaxPool2d(2),
            DualConv(in_channels, out_channels),
        )

    def forward(self, x):
        return self.sequential(x)


class Up(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, bilinear: bool = True):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
            self.conv = DualConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(
                in_channels,
                in_channels // 2,
                kernel_size=2,
                stride=2,
            )
            self.conv = DualConv(in_channels, out_channels)

    def forward(self, x1, x2):
        """
        x1: from the previous layer - decoder
        x2: from the skip connection - encoder
        """
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]  # height
        diffX = x2.size()[3] - x1.size()[3]  # width

        # pad function: (L, R, T, B)
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])

        # Concatenate along the channels axis
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class UNetBaseline(nn.Module):
    def __init__(self, in_channels: int, num_classes: int):
        super(UNetBaseline, self).__init__()

        # Encoder
        # self.inc = DualConv(in_channels, 64)
        # self.down1 = Down(64, 128)
        # self.down2 = Down(128, 256)
        # self.down3 = Down(256, 512)

        # # Bottleneck
        # self.down4 = Down(512, 1024)

        # # Decoder
        # self.up1 = Up(1024, 512, bilinear=False)
        # self.up2 = Up(512, 256, bilinear=False)
        # self.up3 = Up(256, 128, bilinear=False)
        # self.up4 = Up(128, 64, bilinear=False)

        # Output layer
        # self.outc = nn.Conv2d(64, num_classes, kernel_size=1)

        # Smaller UNet for faster training
        self.inc = DualConv(in_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 512)  # Bottleneck
        self.up1 = Up(512, 256, bilinear=False)
        self.up2 = Up(256, 128, bilinear=False)
        self.up3 = Up(128, 64, bilinear=False)
        self.up4 = Up(64, 32, bilinear=False)
        self.outc = nn.Conv2d(32, num_classes, kernel_size=1)

    def forward(self, x):
        # Encoder with skip connections
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)  # Bottleneck

        # Decoder with skip connections
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)

        return x

In [32]:
!cat /proc/meminfo

MemTotal:       13286964 kB
MemFree:          242776 kB
MemAvailable:   11540600 kB
Buffers:          181380 kB
Cached:         10984000 kB
SwapCached:            0 kB
Active:           882396 kB
Inactive:       11477560 kB
Active(anon):       1440 kB
Inactive(anon):  1195176 kB
Active(file):     880956 kB
Inactive(file): 10282384 kB
Unevictable:           8 kB
Mlocked:               8 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               292 kB
Writeback:             0 kB
AnonPages:       1194612 kB
Mapped:           583516 kB
Shmem:              2032 kB
KReclaimable:     460128 kB
Slab:             526480 kB
SReclaimable:     460128 kB
SUnreclaim:        66352 kB
KernelStack:        5424 kB
PageTables:        10220 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6643480 kB
Committed_AS:    3982484 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       91076 kB
VmallocChunk:    

In [48]:
import os
import torch
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from google.colab import files

TRAIN_DATASET_PATH = "./BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"

def pathListIntoIds(dirList):
    x = []
    for i in range(0, len(dirList)):
        x.append(dirList[i][dirList[i].rfind("/") + 1 :])

    return x


def dice_coefficient(pred, target, num_classes=4):
    """
    Calculate Dice Coefficient for each class
    Args:
        pred: (batch, num_classes, H, W) - logits
        target: (batch, num_classes, H, W) - one-hot encoded
        num_classes: number of segmentation classes

    Returns:
        dict: {class_id: dice_score}
    """
    pred = torch.softmax(pred, dim=1)
    pred = torch.argmax(pred, dim=1)  # (batch, H, W)
    target = torch.argmax(target, dim=1)  # (batch, H, W)

    dice_scores = {}
    for class_id in range(num_classes):
        pred_mask = (pred == class_id).float()
        target_mask = (target == class_id).float()

        intersection = (pred_mask * target_mask).sum()
        union = pred_mask.sum() + target_mask.sum()

        if union == 0:
            dice = 1.0  # Perfect score if both are empty
        else:
            dice = (2.0 * intersection) / union

        dice_scores[class_id] = dice.item()  # type: ignore

    return dice_scores


def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    pbar = tqdm(dataloader, desc="Training")
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        avg_loss = running_loss / (batch_idx + 1)

        pbar.set_postfix({"loss": f"{avg_loss:.4f}"})

    return running_loss / len(dataloader)


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_dice_scores = {0: [], 1: [], 2: [], 3: []}

    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Validation")
        for batch_idx, (inputs, targets) in enumerate(pbar):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Calculate metrics
            running_loss += loss.item()
            dice_scores = dice_coefficient(outputs, targets)

            for class_id, score in dice_scores.items():
                all_dice_scores[class_id].append(score)

            avg_loss = running_loss / (batch_idx + 1)
            pbar.set_postfix({"loss": f"{avg_loss:.4f}"})

    # Calculate average dice scores
    avg_dice = {k: np.mean(v) for k, v in all_dice_scores.items()}

    return running_loss / len(dataloader), avg_dice

def train_u_net(
    train_dataset,
    val_dataset,
    num_epochs=50,
    batch_size=8,
    learning_rate=1e-4,
    device="cuda",
    save_dir="checkpoints",
):
    os.makedirs(save_dir, exist_ok=True)
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    # Initialize model
    model = UNetBaseline(in_channels=2, num_classes=4).to(device)

    # Loss and optimizer
    criterion = CombinedLoss(ce_weight=0.5, dice_weight=0.5)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=5,
    )

    # Training history
    history = {"train_loss": [], "val_loss": [], "val_dice": []}
    best_val_loss = float("inf")
    print(f"{'='*70}")
    print(f"Training Configuration:")
    print(f"  Device: {device}")
    print(f"  Epochs: {num_epochs}")
    print(f"  Batch size: {batch_size}")
    print(f"  Learning rate: {learning_rate}")
    print(f"  Train samples: {len(train_dataset)}")
    print(f"  Val samples: {len(val_dataset)}")
    print(f"{'='*70}\n")

    # Training loop
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 70)

        # Train
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)

        # Validate
        val_loss, val_dice = validate(model, val_loader, criterion, device)

        # Update learning rate
        scheduler.step(val_loss)

        # Save history
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_dice"].append(val_dice)

        # Print results
        print(f"\nResults:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Dice Scores:")
        for class_id, score in val_dice.items():
            class_name = ["Background", "Necrotic", "Edema", "Enhancing"][class_id]
            print(f"    Class {class_id} ({class_name}): {score:.4f}")

        print(f"    Mean Dice: {np.mean(list(val_dice.values())):.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_path = os.path.join(save_dir, "best_model.pth")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": val_loss,
                    "val_dice": val_dice,
                },
                save_path,
            )
            print(f"  → Saved best model to {save_path}")

        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch+1}.pth")
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "val_loss": val_loss,
                },
                save_path,
            )
    print(f"\n{'='*70}")
    print("Training completed!")
    print(f"Best validation loss: {best_val_loss:.4f}")
    print(f"{'='*70}")

    return model, history

def download_checkpoints():
    files.download("checkpoints/best_model.pth")

train_and_val_directories = [
    f.path for f in os.scandir(TRAIN_DATASET_PATH) if f.is_dir()
]

train_and_test_ids = pathListIntoIds(train_and_val_directories)
train_test_ids, val_ids = train_test_split(train_and_test_ids, test_size=0.2)
train_ids, test_ids = train_test_split(train_test_ids, test_size=0.15)

train_dataset = BrainTumorDataset(ids=train_ids, data_path=TRAIN_DATASET_PATH)
val_dataset = BrainTumorDataset(ids=val_ids, data_path=TRAIN_DATASET_PATH)

# Train model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, history = train_u_net(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_epochs=50,
    batch_size=8,
    learning_rate=1e-4,
    device=device,
    save_dir="checkpoints",
)

print("\n✓ Training completed!")
download_checkpoints()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training Configuration:
  Device: cuda
  Epochs: 50
  Batch size: 8
  Learning rate: 0.0001
  Train samples: 25000
  Val samples: 7400


Epoch 1/50
----------------------------------------------------------------------


Training:   2%|▏         | 56/3125 [01:06<58:20,  1.14s/it, loss=1.1766]  Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e7024a79620>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Training:  12%|█▏        | 386/3125 [07:28<40:42,  1.12it/s, loss=0.9629]  Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e7024a79620>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/datalo

AttributeError: 'float' object has no attribute 'item'